# __Описание__

Расчетчик, зная табельный номер, филиал и подразделение работника, должен по таблице найти размер его надбавки.
Таблица содержит следующие поля:
- Филиал
- Подразделение
- Табельный номер
- Процент надбавки

# __Задача__

1. Сгенерировать данные
 -  В N филиалах имеется от 1 от N_pod_min до N_pod_max подразделений, в которых работают от N_wor_min до N_wor_max работников. Каждый работник получает надбавку в размере P_n, где n - количество разных надбавок, распределение процентов неравномерно
 -  Известно, что имеются филиалы и подразделения, в которых у всех работников одинаковые проценты надбавки
   
2. Сократить количество строк, так чтобы можно было, зная таблельный номер, филиал и подразделение работника, определить размер его надбавки. В итоговой таблице должны быть только столбцы из исходной таблицы.
 -  Сжимаются до одной строки все филиалы, в которых все работники получают одинаковую надбавку
 -  Сжимаются до одной строки все подразделения, в которых все работники получают одинаковую надбавку
 -  Сжимаются строки с самым частым процентом в подраздлении. Если таких процентов несколько, то сжимаются строки самым большим процентом

In [1]:
import pandas as pd
import random
import numpy as np
import math

# Формирование случайных данных

In [2]:
# Количество филиалов
N = 74
# Максимальное количество подазделений в филиале
N_pod_min = 10
N_pod_max = 54
# Минимальное и максимальное количество работников в подразделении
N_wor_min = 10
N_wor_max = 54
# Количество типов надбавок
n_proc = 4
# доля филиалов с одинаковым процентом
part_uniform_filial = 0.1
# доля подразделений с одинаковым процентом
part_uniform_podrazd = 0.02 

### Создание N филиалов

In [3]:
# Определяем список
my_list = [ 'A', 'E', 'I', 'O', 'U', 'Y', 'B', 'C', 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z']
# Генерируем случайный индекс
filial= set()
while len(filial) < N:
    random_index = ''.join(random.choices(my_list, k=4))
    filial.add(random_index)
filial = list(filial)
df = pd.DataFrame({'filial':filial}).sort_values(by = 'filial')

###  В каждом филиале от 1 до  N_pod позраздлений, которые внутри каждого филиала нумеруется просто по порядку. 

Определение количества подразделений в филиале

In [4]:
n_podrazd = []
for i in filial:
    n = random.randint(N_pod_min, N_pod_max+1)
    n_podrazd.append(n)
n_podrazd = list(n_podrazd)
df['n_podrazd'] = n_podrazd

In [5]:
df = df.loc[df.index.repeat(df['n_podrazd'])].sort_values(by = 'filial')
df.reset_index(drop=True, inplace=True)

Нумерация поздразделений

In [6]:
df['podrazd'] = df.groupby('filial')['n_podrazd'].rank(method='first', ascending=True).astype('int64')
df = df.sort_values(by = ['filial', 'podrazd'])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2227 entries, 0 to 2226
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   filial     2227 non-null   object
 1   n_podrazd  2227 non-null   int64 
 2   podrazd    2227 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 52.3+ KB
None


###  В каждом подразделении от N_wor_min до N_wor_max работников с уникальными номерами

Определим количество работников в подразделениях

In [7]:
n_workers = []
for i in df['podrazd']:
    n = random.randint(N_wor_min,N_wor_max)
    n_workers.append(n)
df['n_workers'] = n_workers

In [8]:
df = df.loc[df.index.repeat(df['n_workers'])]
df.reset_index(drop=True, inplace=True)
df.sort_values(by = ['filial','podrazd']).head(20)

,filial,n_podrazd,podrazd,n_workers
0,AQXZ,55,1,53
1,AQXZ,55,1,53
2,AQXZ,55,1,53
3,AQXZ,55,1,53
4,AQXZ,55,1,53
5,AQXZ,55,1,53
6,AQXZ,55,1,53
7,AQXZ,55,1,53
8,AQXZ,55,1,53
9,AQXZ,55,1,53


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71515 entries, 0 to 71514
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   filial     71515 non-null  object
 1   n_podrazd  71515 non-null  int64 
 2   podrazd    71515 non-null  int64 
 3   n_workers  71515 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 2.2+ MB


Определение табельных номеров

In [10]:
tabel_list = [ '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# Генерируем случайный индекс
tabel = set()
while len(tabel) < len(df['n_workers']):
    random_index = ''.join(random.choices(tabel_list, k=6))
    tabel.add(random_index)

In [11]:
df['tabel'] = list(tabel)
print(len(df['tabel']))
print(df['tabel'].nunique())
print(len(df['filial']))

71515
71515
71515


### Процент надбавки

Формируем список из которого будут выбираться проценты с разными вероятностями

In [12]:
random_proc = []
for i,j in enumerate(range(n_proc)):
    b = random.randint(1,10)
    proc_list = [j] * b
    random_proc.extend(proc_list)
print(random_proc)

[0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]


In [13]:
proc = []
for i in range(len(df['filial'])):
    p = random.choices(random_proc, k=1)
    proc.extend(p)

df['proc'] = proc
df.info()
df.sort_values(by = ['filial','podrazd', 'proc']).head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71515 entries, 0 to 71514
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   filial     71515 non-null  object
 1   n_podrazd  71515 non-null  int64 
 2   podrazd    71515 non-null  int64 
 3   n_workers  71515 non-null  int64 
 4   tabel      71515 non-null  object
 5   proc       71515 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 3.3+ MB


,filial,n_podrazd,podrazd,n_workers,tabel,proc
6,AQXZ,55,1,53,340933,0
16,AQXZ,55,1,53,933214,0
25,AQXZ,55,1,53,147179,0
31,AQXZ,55,1,53,439794,0
28,AQXZ,55,1,53,722288,1
0,AQXZ,55,1,53,836759,2
1,AQXZ,55,1,53,137555,2
2,AQXZ,55,1,53,541994,2
3,AQXZ,55,1,53,317836,2
4,AQXZ,55,1,53,108759,2


In [14]:
df = df.drop(['n_podrazd', 'n_workers'], axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71515 entries, 0 to 71514
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   filial   71515 non-null  object
 1   podrazd  71515 non-null  int64 
 2   tabel    71515 non-null  object
 3   proc     71515 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


### Присвоим работникам некторых филиалов и подразделений одинаковый процент надбавки

Зададим одинаковый процент для всех работников в некотрых случайных филиалах

In [15]:
N = math.ceil(df['filial'].nunique()*part_uniform_filial)
filial_uniform = df['filial'].sample(N , replace = True)
df.loc[df['filial'].isin(filial_uniform),'proc'] = random.choices(range(n), k=1)[0]

Зададим одинаковый процент для всех работников в некотрых случайных подразделениях

In [16]:
N = math.ceil((df['filial']+ df['podrazd'].astype('str')).nunique()*part_uniform_podrazd)
podraz_uniform = df[['filial', 'podrazd']].sample(N , replace = False)
df.loc[(df['filial'].isin(podraz_uniform['filial'])) & (df['podrazd'].isin(podraz_uniform['podrazd'])),'proc'] = random.choices(range(n_proc), k=1)[0]

# Добавим столбцы для определения строк для сжатия

### Вычислим количество работников с соответствующим процентом  в каждом филиале

In [17]:
df['count_proc_filial']=  df.groupby(['filial','proc'])['tabel'].transform('count')

### Вычислим общее количество работников в филиале

In [18]:
df['count_filial']=  df.groupby(['filial'])['tabel'].transform('count')

### Вычислим количество работников с соответствующим процентом  в каждом подразделении.

In [19]:
df['count_proc_podrazd']=  df.groupby(['filial','podrazd','proc'])['tabel'].transform('count')

### Вычислим общее количество работников в каждом подразделении

In [20]:
df['count_podrazd']=  df.groupby(['filial', 'podrazd'])['tabel'].transform('count')

In [21]:
df

,filial,podrazd,tabel,proc,count_proc_filial,count_filial,count_proc_podrazd,count_podrazd
0,AQXZ,1,836759,2,1699,1699,53,53
1,AQXZ,1,137555,2,1699,1699,53,53
2,AQXZ,1,541994,2,1699,1699,53,53
3,AQXZ,1,317836,2,1699,1699,53,53
4,AQXZ,1,108759,2,1699,1699,53,53
...,...,...,...,...,...,...,...,...
71510,ZSGB,43,568791,2,912,1275,32,51
71511,ZSGB,43,251178,2,912,1275,32,51
71512,ZSGB,43,756306,3,90,1275,2,51
71513,ZSGB,43,588059,2,912,1275,32,51


### Определим работников с каким-процентом больше всего в каждом подразделении

In [22]:
df['proc_podrazd_cmax'] = df.groupby(['filial','podrazd'])['count_proc_podrazd'].transform('max')
df['max_proc'] = df.groupby(['filial','podrazd'])['proc'].transform('max')
df

,filial,podrazd,tabel,proc,count_proc_filial,count_filial,count_proc_podrazd,count_podrazd,proc_podrazd_cmax,max_proc
0,AQXZ,1,836759,2,1699,1699,53,53,53,2
1,AQXZ,1,137555,2,1699,1699,53,53,53,2
2,AQXZ,1,541994,2,1699,1699,53,53,53,2
3,AQXZ,1,317836,2,1699,1699,53,53,53,2
4,AQXZ,1,108759,2,1699,1699,53,53,53,2
...,...,...,...,...,...,...,...,...,...,...
71510,ZSGB,43,568791,2,912,1275,32,51,32,3
71511,ZSGB,43,251178,2,912,1275,32,51,32,3
71512,ZSGB,43,756306,3,90,1275,2,51,32,3
71513,ZSGB,43,588059,2,912,1275,32,51,32,3


### Для подразделений, в которых имеется два или более самых распространненых процента, будем ужимать строки с самым большим из них




Проставим самым частым процентам ранги с соответствии  размером процента

In [23]:
df.loc[df['proc_podrazd_cmax'] == df['count_proc_podrazd'], 'rank'] = df.groupby(['filial','podrazd'])['proc'].rank()
df

,filial,podrazd,tabel,proc,count_proc_filial,count_filial,count_proc_podrazd,count_podrazd,proc_podrazd_cmax,max_proc,rank
0,AQXZ,1,836759,2,1699,1699,53,53,53,2,27.0
1,AQXZ,1,137555,2,1699,1699,53,53,53,2,27.0
2,AQXZ,1,541994,2,1699,1699,53,53,53,2,27.0
3,AQXZ,1,317836,2,1699,1699,53,53,53,2,27.0
4,AQXZ,1,108759,2,1699,1699,53,53,53,2,27.0
...,...,...,...,...,...,...,...,...,...,...,...
71510,ZSGB,43,568791,2,912,1275,32,51,32,3,33.5
71511,ZSGB,43,251178,2,912,1275,32,51,32,3,33.5
71512,ZSGB,43,756306,3,90,1275,2,51,32,3,NaN
71513,ZSGB,43,588059,2,912,1275,32,51,32,3,33.5


Найдем для каждого подразделения самый большой ранг процента

In [24]:
df.loc [df['proc_podrazd_cmax'] == df['count_proc_podrazd'], 'rank_max'] = df.groupby(['filial','podrazd'])['rank'].transform('max')
df.shape

(71515, 12)

# Удаление лишних строк

### Сожмем данные о филиалах, в которых у все работников совпадают проценты

In [25]:
df_dupl = df[~((df.duplicated(['filial','proc']))&(df['count_filial'] ==df ['count_proc_filial']))]
df_dupl.shape

(61893, 12)

### Сожмем данные о подразделениях, в которых у всех работников совпадают проценты

In [26]:
df_dupl2 = df_dupl[~((df_dupl.duplicated(['filial','proc', 'podrazd']))&(df_dupl['count_podrazd'] ==df_dupl['count_proc_podrazd']))]
df_dupl2.shape

(43155, 12)

### Сожмем данные с самыми частотными процентами,  если таких процентов несколько, то сожмем строки самым большим процентом из самых частотынх. 

In [27]:
df_dupl3 = df_dupl2[~((df_dupl2.duplicated(['filial','podrazd','proc']))&(df_dupl2['rank'] == df_dupl2['rank_max']))]
df_dupl3.shape

(14227, 12)

In [28]:
# df_dupl3.loc[(df_dupl3['count_filial'] == df_dupl3['count_proc_filial']) | \
#             (df_dupl3['count_podrazd'] == df_dupl3['count_proc_podrazd']) | \
#            ((df_dupl3['rank'] == df_dupl3['rank_max']) & (df_dupl3['count_proc_podrazd'] > 1) ),'tabel'] = 0

In [29]:
df_dupl3.loc[(df_dupl3['count_filial'] == df_dupl3['count_proc_filial']),'tabel'] = 'filial'
df_dupl3.loc[(df_dupl3['count_podrazd'] == df_dupl3['count_proc_podrazd'])  & (df_dupl3['tabel'] != 'filial'),'tabel'] = 'podrazd'

df_dupl3.loc[(df_dupl3['rank'] == df_dupl3['rank_max']) & (df_dupl3['count_proc_podrazd'] > 1) & (df_dupl3['tabel'] != 'filial')  & (df_dupl3['tabel'] != 'podrazd' ),'tabel'] = 'freq_proc'


In [30]:
df_dupl3 = df_dupl3.drop(['max_proc','rank','rank_max', 'proc_podrazd_cmax', 'count_proc_filial', 'count_filial', 'count_proc_podrazd', 'count_podrazd'], axis = 1).sort_values(by=['filial', 'podrazd', 'proc'])

In [31]:
df_dupl3.reset_index(drop=True, inplace=True)

In [32]:
df_dupl3.head(20)

,filial,podrazd,tabel,proc
0,AQXZ,1,filial,2
1,BXAZ,1,306478,0
2,BXAZ,1,964920,0
3,BXAZ,1,513055,0
4,BXAZ,1,383409,0
5,BXAZ,1,997191,1
6,BXAZ,1,989313,1
7,BXAZ,1,720370,1
8,BXAZ,1,101107,1
9,BXAZ,1,883979,1


### Описание результата

1. Расчетчик ищет филиал работника, если строчка с филиалом всего одна, то все работники этого филиала получают указаную надбавку.
2. Если строчек  с этимм филиалом несколько, расчетчик  выбирает дополнительно подразделение, и также, если одна строчка, то все работники подразделения получают указанную надбавку
3. Если строчек с искомым подразделением несколько, то расчетчик ищет табельный номер, если табельный номер находится, то работник получает надбавку указанную напротив его табельного номера, в противном случае, - надбавку из строки, где табельный номер не указан

